# UPDATE 04-12-2022

only 2 points are used for the regression: 100% and 50% (the one with 2 GTs)

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
# read Excel file

source_file_name = "NGCC Performance - with and without Carbon Capture 12 15 2021.xlsx"

dfs = pd.read_excel("../resources/" + source_file_name, sheet_name=None, index_col=0, engine='openpyxl')

# strip whitespace for index
for df in dfs.values():
    df.index = df.index.str.strip()

df_no_carbon_cap = dfs['No Carbon Capture']
df_max_power = dfs['Max Power']
df_max_DAC = dfs['Max DAC']

<!-- The whole running performance is divided into 4 pieces:
- 60%-100%: 2 GT in 60%-100%
- 50%-59%: 2 GT in 50%-60%
- 30%-50% (`0.5.1`): 1 GT in 60%-100%
- 25%-30%: 1 GT in 50%-60% -->

In [25]:
pieces = {
    0: [1, 0.5],
}

In [26]:
def read_idx(idx_list):
    return [50 if i == "0.5.1" else i * 100 for i in idx_list]

## Preprocess results
### Duty of PCC reboiler

In [27]:
reboiler_coef = 2.69 # MMBtu/tonne CO2 captured, from Trimeric
lb_to_tonne = 0.000453592

for i in pieces:
    for j in pieces[i]:
        df_max_power.loc["PCC reboiler duty", j] = df_max_power.loc["CO2 captured by PCC", j] * reboiler_coef * lb_to_tonne

df_max_power.iloc[df_max_power.index.get_loc('PCC reboiler duty'), 0] = "MMBtu/hr"
df_max_power.loc["PCC reboiler duty"]

Unnamed: 1      MMBtu/hr
1             673.857395
0.9           627.899968
0.8           581.842695
0.7           529.348851
0.6           475.988367
0.5           417.996075
0.5.1         336.928698
0.4           290.921347
0.3           237.994184
0.25          208.998038
Name: PCC reboiler duty, dtype: object

### Auxiliary load

In [28]:
aux_rows = ["Circulating Water Pumps",
            "Combustion Turbine Auxiliaries",
            "Condensate Pumps",
            "Cooling Tower Fans",
            "Feedwater Pumps",
            "Ground Water Pumps",
            "Miscellaneous Balance of Plant",
            "SCR",
            "Steam Turbine Auxiliaries",
            "Transformer Losses"
            ]
df_max_power.loc["Total Auxiliary w/o PCC and DAC"] = df_max_power.loc[aux_rows].sum()
df_max_power.iloc[df_max_power.index.get_loc("Total Auxiliary w/o PCC and DAC"), 0] = "kW"
df_max_power.loc["Total Auxiliary w/o PCC and DAC"]

Unnamed: 1              kW
1             17358.919395
0.9           16544.115948
0.8           15558.596325
0.7           14508.690511
0.6           13509.488383
0.5           12266.054229
0.5.1          9995.709746
0.4            9283.309286
0.3            8430.369506
0.25           7896.148618
Name: Total Auxiliary w/o PCC and DAC, dtype: object

### DAC base steam (energy (MMBtu) based)

In [29]:
hv_hf = 959 # Btu/lb for 20.8 psia steam

for i in pieces:
    for j in pieces[i]:
        df_max_power.loc["DAC base steam", j] = df_max_power.loc["DAC Steam Flow", j] * hv_hf / 1e6

df_max_power.iloc[df_max_power.index.get_loc("DAC base steam"), 0] = "MMBtu/hr"
df_max_power.loc["DAC base steam"]

Unnamed: 1      MMBtu/hr
1             227.382736
0.9           216.203673
0.8            203.00112
0.7           188.195119
0.6           173.247186
0.5            151.68503
0.5.1         119.220003
0.4           106.297478
0.3            90.148877
0.25           77.016331
Name: DAC base steam, dtype: object

### DAC side steam (energy (MMBtu) based)

In [30]:
for i in pieces:
    for j in pieces[i]:
        df_max_power.loc["DAC side steam", j] = (df_max_DAC.loc["DAC Steam Flow", j] - df_max_power.loc["DAC Steam Flow", j]) * hv_hf / 1e6

df_max_power.iloc[df_max_power.index.get_loc("DAC side steam"), 0] = "MMBtu/hr"
df_max_power.loc["DAC side steam"]

Unnamed: 1      MMBtu/hr
1             703.431295
0.9           691.991384
0.8           658.870401
0.7           627.162984
0.6           599.491039
0.5           555.731869
0.5.1         345.650452
0.4           326.106032
0.3           296.523759
0.25          268.350257
Name: DAC side steam, dtype: object

### CO2 emission

In [31]:
df_no_carbon_cap.loc["CO2 Emission", 1:11] = df_no_carbon_cap.iloc[51, 1:11].multiply(df_no_carbon_cap.iloc[49, 1:11])

df_no_carbon_cap.iloc[df_no_carbon_cap.index.get_loc("CO2 Emission"), 0] = "tonne CO2/h"
df_no_carbon_cap.loc["CO2 Emission", :]

C:\Users\pcheng45\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:719: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  indexer = self._get_setitem_indexer(key)


Unnamed: 1     tonne CO2/h
1               258.256728
0.9             240.643484
0.8             222.991974
0.7             202.873639
0.6             182.423164
0.5             160.197542
0.5.1           129.128364
0.4             111.495987
0.3              91.211582
0.25             80.098771
Unnamed: 12            NaN
Unnamed: 13            NaN
Unnamed: 14            NaN
Name: CO2 Emission, dtype: object

## Regression

In [32]:
params = {
    "GT_power": [df_max_power, "Gas Turbine", "kW"],
    "fuel": [df_max_power, "Heat Input to Turbine(s)", "MMBtu/hr"],
    "HP_power": [df_max_power, "HP", "kW"],
    "IP_power": [df_max_power, "IP", "kW"],
    # "LP_power": [df_max_power, "LP", "kW"],
    "PCC_reboiler_duty": [df_max_power, "PCC reboiler duty", "MMBtu/hr"],
    "auxiliary_power": [df_max_power, "Total Auxiliary w/o PCC and DAC", "MMBtu/hr"],
    "DAC_base_steam_duty": [df_max_power, "DAC base steam", "MMBtu/hr"],
    "DAC_side_steam_duty": [df_max_power, "DAC side steam", "MMBtu/hr"],
    "CO2_emission": [df_no_carbon_cap, "CO2 Emission", "t CO2/hr"]
}

In [33]:
def linear_regress(df, idx_name, piece_idx):
    """
    Conduct linear regression on pieces[piece_idx] of df.idx_name.
    """

    X = np.array(read_idx(pieces[piece_idx])).reshape(-1, 1)
    y = df.loc[idx_name, pieces[piece_idx]].values

    linear_regressor = LinearRegression()
    linear_regressor.fit(X, y)
    return linear_regressor.coef_[0], linear_regressor.intercept_

In [34]:
coefs = []

for k, v in params.items():
    for piece_idx in pieces.keys():
        m, b = linear_regress(v[0], v[1], piece_idx)
        coefs.append([k, m, b, v[2], piece_idx])
        # print(k, m, b, piece_idx)

In [36]:
df_coef = pd.DataFrame(coefs, columns=['y', 'a', 'b', 'unit', 'piece_idx'])
df_coef

,y,a,b,unit,piece_idx
0,GT_power,4770.000000,5.820766e-11,kW,0
1,fuel,35.124399,1.103131e+03,MMBtu/hr,0
2,HP_power,388.180893,1.319196e+04,kW,0
3,IP_power,464.880000,2.267000e+03,kW,0
4,PCC_reboiler_duty,5.117226,1.621348e+02,MMBtu/hr,0
5,auxiliary_power,101.857303,7.173189e+03,MMBtu/hr,0
6,DAC_base_steam_duty,1.513954,7.598732e+01,MMBtu/hr,0
7,DAC_side_steam_duty,2.953989,4.080324e+02,MMBtu/hr,0
8,CO2_emission,1.961184,6.213836e+01,t CO2/hr,0


In [37]:
df_coef.to_csv("coefs.csv", index=False)

### Coefficients from LP steam to power

In [15]:
X = np.array(df_max_power.loc["DAC side steam"].iloc[1:].values).reshape(-1, 1)
y = df_max_power.loc["LP"].iloc[1:].values

linear_regressor = LinearRegression(fit_intercept=False)
linear_regressor.fit(X, y)
linear_regressor.coef_[0] / 1000 # MW / MMBtu/hr

0.09061848382623319